In [7]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()
from pyspark.sql.functions import col, split

# Set-up bucket and spark

Check bucket content.

In [2]:
spark = SparkSession \
    .builder \
    .appName("computer_vision") \
    .master("local") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .config("google.cloud.auth.service.account.json.keyfile", "/path/to/json") \
    .getOrCreate()
    #.config ("spark.hadoop.fs.gs.project.id", "oceanic-hold-342316") \
    #.config("spark.hadoop.fs.gs.auth.service.account.email", "email") \
    #.config("spark.hadoop.fs.gs.auth.service.account.private.key.id", "key_id") \
    #.config("spark.hadoop.fs.gs.auth.service.account.private.key", "key") \

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [4]:
#!gsutil ls gs://p8_computer_vision/img/*

In [6]:
path_img = "gs://p8_computer_vision/img/*"
img_all = spark.read.format("image").load(path_img)
img_all.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [21]:
img_df = img_all.select("image.origin", "image.width", "image.height", "image.data")
img_df = img_df.withColumn("target", split(col("origin"), "/") \
    .getItem(4))
img_df.show()

+--------------------+-----+------+--------------------+------------------+
|              origin|width|height|                data|            target|
+--------------------+-----+------+--------------------+------------------+
|gs://p8_computer_...|  100|   100|[FB FF FF FB FF F...|apple_crimson_snow|
|gs://p8_computer_...|  100|   100|[FD FE FF FD FE F...|apple_crimson_snow|
|gs://p8_computer_...|  100|   100|[FD FF FF FD FF F...|apple_crimson_snow|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|           avocado|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|           avocado|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|           avocado|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|            banana|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|            banana|
|gs://p8_computer_...|  100|   100|[FF FF FF FF FF F...|            banana|
+--------------------+-----+------+--------------------+------------------+

